### Step 1: Import packages

In [ ]:
# install syft if we are on google colab
import sys, os; os.system("pip install --pre syft") if "google.colab" in sys.modules else ""
import syft as sy
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
from utils import *

### Step 2: Add DS Credentials

In [ ]:
domain_credentials = add_credentials(
    [
        {'name': 'Sam Carter', 'email': 'sam@stargate.net', 'password': 'changethis', 'url': 'localhost', "budget": 999},
    ]
)

### Step: 3 Login into the Domains

In [ ]:
domains = login_to_domains(domain_credentials)

### Step 4: View Dataset on one of the Domains

In [ ]:
domains[0].datasets

### Step 5: View Assets on the Dataset

In [ ]:
domains[0].datasets[-1]

### Step 6: Preprocess Images

In [ ]:
X_train , X_val, Y_train, Y_val = preprocess_data_on_domains(domains)

### Step 7: View Synthetic data

In [ ]:
(X_train[0], Y_train[0]), (X_val[0], Y_val[0])

### Step 8: Train Model

In [ ]:
def train_model(X, Y, alpha, iterations):
    label_size = int(Y.max_vals.data-Y.min_vals.data+1)
    m, n = X.public_shape
    W1, b1, W2, b2 = init_params(m, label_size)
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

    return W1, b1, W2, b2

# Contains the weight from all domain nodes
W1, b1, W2, b2 = [], [], [], []
for X, Y in zip(X_train, Y_train):
    W1_train, b1_train, W2_train, b2_train = train_model(X, Y, 0.10, 1)
    
    W1.append(W1_train)
    b1.append(b1_train)
    W2.append(W2_train)
    b2.append(b2_train)

### Step 9: Check status of training

In [ ]:
for p1,p2,p3,p4 in zip(W1,b1,W2,b2):
    p1.block_with_timeout(60)
    p2.block_with_timeout(60)
    p3.block_with_timeout(60)
    p4.block_with_timeout(60)

    print(f"Training Successful on  Domain Client ✅:{p1.client} ")

### Step 10: SMPC Averaging of the Weights

In [ ]:
avg_W1, avg_b1, avg_W2, avg_b2 = smpc_weight_averaging(W1, b1, W2, b2)
for ptr in [avg_W1,avg_b1, avg_W2, avg_b2]:
    ptr.block_with_timeout(60)
    print("SMPC Averaging Successful  ✅")

### Step 11: Publish Weights using DP

In [ ]:
#Publish Weights Using DP
sigma = 100
avg_W1 = avg_W1.publish(sigma=sigma)
avg_b1 = avg_b1.publish(sigma=sigma)
avg_W2 = avg_W2.publish(sigma=sigma)
avg_b2 = avg_b2.publish(sigma=sigma)
print("You have witnessed and trained one of a kind ML Model Training with SMPC +DP ")

### Step 12: Check if public Weights are ready 

In [ ]:
for p1,p2,p3,p4 in [(avg_W1, avg_b1, avg_W2, avg_b2)]:
    p1.block_with_timeout(60)
    p2.block_with_timeout(60)
    p3.block_with_timeout(60)
    p4.block_with_timeout(60)    
print("Results are ready for download !!!")

### Step 13: Download the Weights

In [ ]:
pub_W1, pub_b1, pub_W2, pub_b2 = avg_W1.get_copy(), avg_b1.get_copy(), avg_W2.get_copy(), avg_b2.get_copy()
print("Woohoo.. your results are ready !!!")

### Step 14: View Results

In [ ]:
print(pub_W1, pub_b1, pub_W2, pub_b2)